In [1]:
import pandas as pd
import fakeEDA
from tqdm import tqdm
from fakeEDA import eda
import random
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/hu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
eda('trim so that we have the desired number of augmented sentences')

['trim so that we have the desired number of augmented sentences',
 'trim so that we have the desired number of augmented sentences',
 'trim so that we have the desired number of augmented sentences',
 'trim so that we have the desired number of augmented sentences']

In [3]:
df = pd.read_json('data/cleaned_labeled_data_v3.json')

In [4]:
df.head(10)

,category,headline,description
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,She left her husband. He killed their children...
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Of course it has a song.
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,The actor and his longtime girlfriend Anna Ebe...
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,The actor gives Dems an ass-kicking for not fi...
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,"The ""Dietland"" actress said using the bags is ..."
5,ENTERTAINMENT,Morgan Freeman 'Devastated' That Sexual Harass...,"""It is not right to equate horrific incidents ..."
6,ENTERTAINMENT,What To Watch On Amazon Prime That’s New This ...,There's a great mini-series joining this week.
7,ENTERTAINMENT,Mike Myers Reveals He'd 'Like To' Do A Fourth ...,"Myer's kids may be pushing for a new ""Powers"" ..."
8,ENTERTAINMENT,What To Watch On Hulu That’s New This Week,You're getting a recent Academy Award-winning ...
9,ENTERTAINMENT,Justin Timberlake Visits Texas School Shooting...,"The pop star also wore a ""Santa Fe Strong"" shi..."


In [5]:
content = df['description']
headline = df['headline']
category = df['category']

In [6]:
category_dic = {}
for idx,cate in enumerate(list(set(df['category']))):
    category_dic[cate] = idx

In [7]:
category_dic

{'ENTERTAINMENT': 0,
 'HOME & LIVING': 1,
 'WEIRD NEWS': 2,
 'SCIENCE & TECH': 3,
 'WORLD NEWS': 4,
 'MEDIA': 5,
 'POLITICS': 6,
 'ENVIRONMENT': 7,
 'TRAVEL': 8,
 'FOOD & DRINK': 9,
 'CRIME': 10,
 'ARTS & CULTURE': 11,
 'STYLE & BEAUTY': 12,
 'COMEDY': 13,
 'WELLNESS': 14,
 'RELIGION': 15,
 'DIVORCE': 16,
 'WEDDINGS': 17,
 'IMPACT': 18,
 'BUSINESS & FINANCES': 19,
 'SPORTS': 20,
 'GROUPS VOICES': 21,
 'MISCELLANEOUS': 22,
 'WOMEN': 23,
 'PARENTING': 24,
 'EDUCATION': 25}

In [8]:
total = 0
maxl,minl = 0,100000
dic_count = []
for ca in category_dic.keys():
    cur_len = len([i for i in category if i==ca])
    print('{}:{}'.format(ca,cur_len))
    maxl,minl = max(maxl,cur_len),min(minl,cur_len)
    dic_count.append(cur_len)
    total += cur_len
print('total:',total)
print('max_cate:{}\nmin_cate:{}'.format(maxl,minl))
print(dic_count)

ENTERTAINMENT:11626
HOME & LIVING:3946
WEIRD NEWS:1679
SCIENCE & TECH:3534
WORLD NEWS:6842
MEDIA:2174
POLITICS:27829
ENVIRONMENT:3309
TRAVEL:9218
FOOD & DRINK:6816
CRIME:2579
ARTS & CULTURE:3095
STYLE & BEAUTY:10541
COMEDY:3734
WELLNESS:22639
RELIGION:1798
DIVORCE:3406
WEDDINGS:3632
IMPACT:3032
BUSINESS & FINANCES:6654
SPORTS:3703
GROUPS VOICES:10242
MISCELLANEOUS:1962
WOMEN:2871
PARENTING:11804
EDUCATION:1784
total: 170449
max_cate:27829
min_cate:1679
[11626, 3946, 1679, 3534, 6842, 2174, 27829, 3309, 9218, 6816, 2579, 3095, 10541, 3734, 22639, 1798, 3406, 3632, 3032, 6654, 3703, 10242, 1962, 2871, 11804, 1784]


#### find a proper increase num_aug

In [9]:
for ca in category_dic.keys():
    p = 4/(dic_count[category_dic[ca]]/(sum(dic_count)/len(dic_count)))
    print(category_dic[ca],dic_count[category_dic[ca]],p)

0 11626 2.2555412933875
1 3946 6.645444266833015
2 1679 15.618179319191828
3 3534 7.420181968569066
4 6842 3.832640028781508
5 2174 12.0620621328993
6 27829 0.942287652338319
7 3309 7.924727433340308
8 9218 2.844751906804413
9 6816 3.8472598410978693
10 2579 10.167864706057804
11 3095 8.472673045855599
12 10541 2.4877073405676007
13 3734 7.022743191463063
14 22639 1.158307481643318
15 1798 14.58449559339437
16 3406 7.69903789692398
17 3632 7.219967807522873
18 3032 8.64872133143901
19 6654 3.9409262213590437
20 3703 7.081534722366481
21 10242 2.5603322668349033
22 1962 13.365404218615229
23 2871 9.133724513034858
24 11804 2.221528556160884
25 1784 14.698947913073473


In [10]:
augumented_news = [[],[],[]]

In [11]:
for cate,head,cont in tqdm(zip(category,headline,content)):
    if max(len(head),len(cont)>10):
        p = 4/(dic_count[category_dic[ca]]/(sum(dic_count)/len(dic_count)))
        if len(head)>len(cont):
            new_heads = eda(head,num_aug=max(6,int(p)))
            for new in new_heads:
                augumented_news[0].append(cate)
                augumented_news[1].append(new)
                augumented_news[2].append(cont)
        else:
            new_conts = eda(cont)
            for new in new_conts:
                augumented_news[0].append(cate)
                augumented_news[1].append(head)
                augumented_news[2].append(new)
    else:
        augumented_news[0].append(cate)
        augumented_news[1].append(head)
        augumented_news[2].append(cont)

170449it [00:33, 5146.56it/s]


In [12]:
len(augumented_news[0])

956966

In [13]:
total = 0
maxl,minl = 0,100000
dic_count = []
for ca in category_dic.keys():
    cur_len = len([i for i in augumented_news[0] if i==ca])
    print('{}:{}'.format(ca,cur_len))
    maxl,minl = max(maxl,cur_len),min(minl,cur_len)
    dic_count.append(cur_len)
    total += cur_len
print('total:',total)
print('max_cate:{}\nmin_cate:{}'.format(maxl,minl))
print(dic_count)

ENTERTAINMENT:95634
HOME & LIVING:23834
WEIRD NEWS:15596
SCIENCE & TECH:19096
WORLD NEWS:37978
MEDIA:13636
POLITICS:177066
ENVIRONMENT:15426
TRAVEL:39862
FOOD & DRINK:39564
CRIME:15446
ARTS & CULTURE:17080
STYLE & BEAUTY:54804
COMEDY:30476
WELLNESS:97706
RELIGION:9502
DIVORCE:13794
WEDDINGS:14708
IMPACT:14568
BUSINESS & FINANCES:30206
SPORTS:26872
GROUPS VOICES:61538
MISCELLANEOUS:12138
WOMEN:17124
PARENTING:54976
EDUCATION:8336
total: 956966
max_cate:177066
min_cate:8336
[95634, 23834, 15596, 19096, 37978, 13636, 177066, 15426, 39862, 39564, 15446, 17080, 54804, 30476, 97706, 9502, 13794, 14708, 14568, 30206, 26872, 61538, 12138, 17124, 54976, 8336]


In [14]:
shuffle_idx = [i for i in range(0,len(augumented_news[0]))]
random.shuffle(shuffle_idx)
shuffled_data = [[],[],[]]
for idx in shuffle_idx:
    shuffled_data[0].append(augumented_news[0][idx])
    shuffled_data[1].append(augumented_news[1][idx])
    shuffled_data[2].append(augumented_news[2][idx])
    

In [15]:
augmented_df = {
    'category':shuffled_data[0],
    'headline':shuffled_data[1],
    'description':shuffled_data[2]
}

In [16]:
augmented_df = pd.DataFrame(data=augmented_df)

In [17]:
augmented_df['category'].head(10)

0               POLITICS
1         STYLE & BEAUTY
2    BUSINESS & FINANCES
3                 COMEDY
4               POLITICS
5          ENTERTAINMENT
6             WORLD NEWS
7                  CRIME
8               WELLNESS
9          ENTERTAINMENT
Name: category, dtype: object

In [18]:
augmented_df.to_json('data/fake-augmented_labeled_news.json')